In [1]:
import numpy as np 
import pandas as pd 

from scipy import stats
import math

from string import punctuation
import re

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.linear_model import Lasso, Ridge, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import lightgbm as lgb

import warnings

pd.set_option('max_columns', 200)
pd.set_option('max_rows', 80)

In [2]:
def cv_score(df_train, y_train, kfolds, pipeline, imp_coef=False):
    oof = np.zeros(len(df_train))
    train = df_train.copy()
    
    feat_df = pd.DataFrame()
    
    for n_fold, (train_index, test_index) in enumerate(kfolds.split(train.values)):
            
        trn_data = train.iloc[train_index][:]
        val_data = train.iloc[test_index][:]
        
        trn_target = y_train.iloc[train_index].values.ravel()
        val_target = y_train.iloc[test_index].values.ravel()
        
        pipeline.fit(trn_data, trn_target)

        oof[test_index] = pipeline.predict(val_data).ravel()

        if imp_coef:
            try:
                fold_df = get_coef(pipeline)
            except AttributeError:
                fold_df = get_feature_importance(pipeline)
                
            fold_df['fold'] = n_fold + 1
            feat_df = pd.concat([feat_df, fold_df], axis=0)
       
    if imp_coef:
        feat_df = feat_df.groupby('feat')['score'].agg(['mean', 'std'])
        feat_df['abs_sco'] = (abs(feat_df['mean']))
        feat_df = feat_df.sort_values(by=['abs_sco'],ascending=False)
        del feat_df['abs_sco']
        return oof, feat_df
    else:    
        return oof
    

def get_coef(pipe):
    imp = pipe.steps[-1][1].coef_.tolist()
    feats = pipe.steps[-2][1].get_feature_names()
    result = pd.DataFrame({'feat':feats,'score':imp})
    result['abs_res'] = abs(result['score'])
    result = result.sort_values(by=['abs_res'],ascending=False)
    del result['abs_res']
    return result


def get_feature_importance(pipe):
    imp = pipe.steps[-1][1].feature_importances_.tolist() #it's a pipeline
    feats = pipe.steps[-2][1].get_feature_names()
    result = pd.DataFrame({'feat':feats,'score':imp})
    result = result.sort_values(by=['score'],ascending=False)
    return result

In [3]:
def defense_formation(l):
    dl = 0
    lb = 0
    db = 0
    other = 0

    for position in l:
        sub_string = position.split(' ')
        if sub_string[1] == 'DL':
            dl += int(sub_string[0])
        elif sub_string[1] in ['LB','OL']:
            lb += int(sub_string[0])
        else:
            db += int(sub_string[0])

    counts = (dl,lb,db,other)

    return counts


def offense_formation(l):
    qb = 0
    rb = 0
    wr = 0
    te = 0
    ol = 0

    sub_total = 0
    qb_listed = False
    for position in l:
        sub_string = position.split(' ')
        pos = sub_string[1]
        cnt = int(sub_string[0])

        if pos == 'QB':
            qb += cnt
            sub_total += cnt
            qb_listed = True
        # Assuming LB is a line backer lined up as full back
        elif pos in ['RB','LB']:
            rb += cnt
            sub_total += cnt
        # Assuming DB is a defensive back and lined up as WR
        elif pos in ['WR','DB']:
            wr += cnt
            sub_total += cnt
        elif pos == 'TE':
            te += cnt
            sub_total += cnt
        # Assuming DL is a defensive lineman lined up as an additional line man
        else:
            ol += cnt
            sub_total += cnt

    # If not all 11 players were noted at given positions we need to make some assumptions
    # I will assume if a QB is not listed then there was 1 QB on the play
    # If a QB is listed then I'm going to assume the rest of the positions are at OL
    # This might be flawed but it looks like RB, TE and WR are always listed in the personnel
    if sub_total < 11:
        diff = 11 - sub_total
        if not qb_listed:
            qb += 1
            diff -= 1
        ol += diff

    counts = (qb,rb,wr,te,ol)

    return counts

In [4]:
class transformation(TransformerMixin, BaseEstimator):
    def __init__(self, mean_weight=10):
        self.columns = None
        self.mean_weight = mean_weight
        self.smooth_team = {}
    
    
    def smooth_te(self, data, target, col):
        tmp_data = data.copy()
        tmp_data['target'] = target
        mean_tot = tmp_data['target'].mean()
        means = tmp_data.groupby(col)['target'].mean()
        counts = tmp_data.groupby(col)['target'].count()

        smooth = ((counts * means + self.mean_weight * mean_tot) / 
                       (counts + self.mean_weight))
        return mean_tot, smooth

        
    def fit(self, X, y=None):
        return self
    
    
    def stats_by_play(self, data):
        avg_by_play = data.groupby(['PlayId', 
                                    'Team', 
                                    'offense_team'], as_index=False)[['PlayerHeight', 
                                                                      'PlayerWeight',
                                                                      'age',
                                                                      'S', 'A']].mean()
        spread = data.groupby(['PlayId', 
                               'Team', 
                               'offense_team'])[['X', 'Y']].std().reset_index()
        tot_momentum = data.groupby(['PlayId', 
                                     'Team', 
                                     'offense_team'], as_index=False)[['X_speed', 'Y_speed',
                                                                       'PlayerWeight',
                                                                       'X_acceleration','Y_acceleration']].sum()
        
        tot_momentum['x_momentum'] = tot_momentum['X_speed'] * tot_momentum['PlayerWeight']
        tot_momentum['y_momentum'] = tot_momentum['Y_speed'] * tot_momentum['PlayerWeight']
        tot_momentum['x_force'] = tot_momentum['X_acceleration'] * tot_momentum['PlayerWeight']
        tot_momentum['y_force'] = tot_momentum['Y_acceleration'] * tot_momentum['PlayerWeight']
        tot_momentum.drop(['X_speed', 'Y_speed',
                           'PlayerWeight',  
                           'X_acceleration','Y_acceleration'], axis=1, inplace=True)

        avg_by_play = pd.merge(avg_by_play, tot_momentum, on=['PlayId', 'Team', 'offense_team'])
        avg_by_play = pd.merge(avg_by_play, spread, on=['PlayId', 'Team', 'offense_team'])

        poss_team = avg_by_play[avg_by_play.Team == avg_by_play.offense_team].copy()
        def_team = avg_by_play[avg_by_play.Team != avg_by_play.offense_team].copy()

        poss_team.rename(columns={'PlayerHeight': 'poss_avg_height', 
                                  'PlayerWeight': 'poss_avg_weight',
                                  'age': 'poss_avg_age',
                                  'X': 'poss_std_X',
                                  'Y': 'poss_std_Y',
                                  'S': 'poss_avg_S', 
                                  'A': 'poss_avg_A', 
                                  'x_momentum': 'poss_x_momentum', 
                                  'y_momentum': 'poss_y_momentum', 
                                  'x_force': 'poss_x_force', 
                                  'y_force': 'poss_y_force'}, inplace=True)
        def_team.rename(columns={'PlayerHeight': 'def_avg_height', 
                                  'PlayerWeight': 'def_avg_weight', 
                                  'age': 'def_avg_age',
                                  'X': 'def_std_X',
                                  'Y': 'def_std_Y',
                                  'S': 'def_avg_S', 
                                  'A': 'def_avg_A',
                                  'x_momentum': 'def_x_momentum', 
                                  'y_momentum': 'def_y_momentum', 
                                  'x_force': 'def_x_force', 
                                  'y_force': 'def_y_force'}, inplace=True)

        avg_by_play = pd.merge(poss_team.drop('Team', axis=1), 
                               def_team.drop('Team', axis=1), on=['PlayId', 'offense_team'])
        
        avg_by_play['tot_x_momenumt'] = avg_by_play['poss_x_momentum'] - avg_by_play['def_x_momentum']
        avg_by_play['tot_x_force'] = avg_by_play['poss_x_force'] - avg_by_play['def_x_force']
        avg_by_play['height_diff'] = avg_by_play['poss_avg_height'] - avg_by_play['def_avg_height']
        avg_by_play['weight_diff'] = avg_by_play['poss_avg_weight'] - avg_by_play['def_avg_weight']
        avg_by_play['age_diff'] = avg_by_play['poss_avg_age'] - avg_by_play['def_avg_age']
        avg_by_play['X_diff'] = avg_by_play['poss_std_X'] - avg_by_play['def_std_X']
        avg_by_play['Y_diff'] = avg_by_play['poss_std_Y'] - avg_by_play['def_std_Y']

        return avg_by_play
    
    
    def personnel_features(self, X):
        personnel = X[['PlayId','OffensePersonnel','DefensePersonnel']].drop_duplicates()
        personnel['DefensePersonnel'] = personnel['DefensePersonnel'].str.split(', ')
        personnel['DefensePersonnel'] = personnel['DefensePersonnel'].apply(lambda x: defense_formation(x))
        personnel['num_DL'] = personnel['DefensePersonnel'].apply(lambda x: x[0])
        personnel['num_LB'] = personnel['DefensePersonnel'].apply(lambda x: x[1])
        personnel['num_DB'] = personnel['DefensePersonnel'].apply(lambda x: x[2])

        personnel['OffensePersonnel'] = personnel['OffensePersonnel'].str.split(', ')
        personnel['OffensePersonnel'] = personnel['OffensePersonnel'].apply(lambda x: offense_formation(x))
        personnel['num_QB'] = personnel['OffensePersonnel'].apply(lambda x: x[0])
        personnel['num_RB'] = personnel['OffensePersonnel'].apply(lambda x: x[1])
        personnel['num_WR'] = personnel['OffensePersonnel'].apply(lambda x: x[2])
        personnel['num_TE'] = personnel['OffensePersonnel'].apply(lambda x: x[3])
        personnel['num_OL'] = personnel['OffensePersonnel'].apply(lambda x: x[4])

        # Let's create some features to specify if the OL is covered
        personnel['OL_diff'] = personnel['num_OL'] - personnel['num_DL']
        personnel['OL_TE_diff'] = (personnel['num_OL'] + personnel['num_TE']) - personnel['num_DL']
        # Let's create a feature to specify if the defense is preventing the run
        # Let's just assume 7 or more DL and LB is run prevention
        personnel['run_def'] = (personnel['num_DL'] + personnel['num_LB'] > 6).astype(int)

        personnel.drop(['OffensePersonnel','DefensePersonnel'], axis=1, inplace=True)

        return personnel
    
    
    def process_play(self, X):
        cols_by_play = ['GameId', 'PlayId', 'YardLine', 
                'Quarter', 'GameClock', 'Down', 'Distance',
                'OffenseFormation', 'DefendersInTheBox',  
                'Yards','Location', 'StadiumType', 'Turf', 
                'GameWeather','Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 
                'PlayDirection', 'HomeScoreBeforePlay', 'VisitorScoreBeforePlay']
        train_play = X[cols_by_play].drop_duplicates()
        avg_by_play = self.stats_by_play(X)
        personnel = self.personnel_features(X)
        train_play = pd.merge(train_play, avg_by_play.drop('offense_team', axis=1), on=['PlayId'])
        train_play = pd.merge(train_play, personnel, on=['PlayId'])

        return train_play
    
    
    def transform(self, X, y=None):
        train_play = self.process_play(X)
        carriers = X[X.has_ball].copy()

        to_drop = ['GameId', 'NflId', 'Team', 'Orientation','YardLine', 'Quarter', 'GameClock', 'PossessionTeam',
           'Down', 'FieldPosition', 'HomeScoreBeforePlay',
           'VisitorScoreBeforePlay', 'NflIdRusher', 'OffensePersonnel','DefensePersonnel',
               'PlayDirection', 'Yards', 'Position', 'HomeTeamAbbr',
           'VisitorTeamAbbr', 'Location', 'StadiumType', 'GameWeather',
           'Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 'to_left',
           'has_ball', 'offense_team', 'Distance',
           'OffenseFormation', 'DefendersInTheBox', 'Turf']

        carriers.drop(to_drop, axis=1, inplace=True)

        full_train = pd.merge(carriers, train_play, on='PlayId')

        full_train.drop(['GameId', 'WindDirection', 'WindSpeed', 'GameWeather', 
                         'PlayDirection', 'StadiumType', 'Turf', 'Location', 'GameClock'], axis=1, inplace=True)
        
        self.columns = full_train.columns

        return full_train
    
    
    def get_features_name(self):
        return self.columns

In [5]:
class df_imputer(TransformerMixin, BaseEstimator):
    '''
    Just a wrapper for the SimpleImputer that keeps the dataframe structure
    '''
    def __init__(self, strategy='mean'):
        self.strategy = strategy
        self.imp = None
        self.statistics_ = None

    def fit(self, X, y=None):
        self.imp = SimpleImputer(strategy=self.strategy)
        self.imp.fit(X)
        self.statistics_ = pd.Series(self.imp.statistics_, index=X.columns)
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        Ximp = self.imp.transform(X)
        Xfilled = pd.DataFrame(Ximp, index=X.index, columns=X.columns)
        return Xfilled

    
class df_scaler(TransformerMixin, BaseEstimator):
    '''
    Wrapper of StandardScaler or RobustScaler
    '''
    def __init__(self, method='standard'):
        self.scl = None
        self.scale_ = None
        self.method = method
        if self.method == 'sdandard':
            self.mean_ = None
        elif method == 'robust':
            self.center_ = None
        self.columns = None  # this is useful when it is the last step of a pipeline before the model

    def fit(self, X, y=None):
        if self.method == 'standard':
            self.scl = StandardScaler()
            self.scl.fit(X)
            self.mean_ = pd.Series(self.scl.mean_, index=X.columns)
        elif self.method == 'robust':
            self.scl = RobustScaler()
            self.scl.fit(X)
            self.center_ = pd.Series(self.scl.center_, index=X.columns)
        self.scale_ = pd.Series(self.scl.scale_, index=X.columns)
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        Xscl = self.scl.transform(X)
        Xscaled = pd.DataFrame(Xscl, index=X.index, columns=X.columns)
        self.columns = X.columns
        return Xscaled

    def get_feature_names(self):
        return list(self.columns)


class dummify(TransformerMixin, BaseEstimator):
    '''
    Wrapper for get dummies
    Via match_cols, it is possible to ask the transformer to make sure that all the dummies are there
    Missing dummies are introduced with a column of 0's
    Extra dummies are dropped
    '''
    def __init__(self, drop_first=False, match_cols=True):
        self.drop_first = drop_first
        self.columns = []  # useful to well behave with FeatureUnion
        self.match_cols = match_cols

    def fit(self, X, y=None):
        return self
    
    def match_columns(self, X):
        miss_train = list(set(X.columns) - set(self.columns))
        miss_test = list(set(self.columns) - set(X.columns))
        
        err = 0

        if len(miss_test) > 0:
            for col in miss_test:
                X[col] = 0  # insert a column for the missing dummy
                err += 1
        if len(miss_train) > 0:
            for col in miss_train:
                del X[col]  # delete the column of the extra dummy
                err += 1
                
        if err > 0:
            warnings.warn('The dummies in this set do not match the ones in the train set, we corrected the issue.',
                         UserWarning)
            
        return X[self.columns]
        
    def transform(self, X):
        X = pd.get_dummies(X, drop_first=self.drop_first)
        if (len(self.columns) > 0):
            if self.match_cols:
                X = self.match_columns(X)
        else:
            self.columns = X.columns
        return X
    
    def get_features_name(self):
        return list(self.columns)

In [6]:
def create_targets(data):
    unique_plays = data[['PlayId', 'Yards']].drop_duplicates()
    simple = unique_plays['Yards'].reset_index(drop=True)
    # As total distance of the rusher
    rushers = data[data.has_ball].copy().reset_index(drop=True)
    tot_dist = rushers['Yards'] + rushers['from_yardline']
    # As percentage of Yards remaining to be gained
    unique_plays = data[['PlayId', 'YardLine', 'Yards']].drop_duplicates().reset_index(drop=True)
    perc_gained = unique_plays['Yards'] / (110 - unique_plays['YardLine'])
    # As both
    perc_dist = tot_dist / (110 - rushers['X'])
    
    return simple, tot_dist, perc_gained, perc_dist

In [7]:
df_train = pd.read_csv('../data_processed/train_processed.csv', dtype={'WindSpeed': 'object'})

df_train.head()

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,Yards,PlayerHeight,PlayerWeight,Position,HomeTeamAbbr,VisitorTeamAbbr,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection,to_left,has_ball,offense_team,from_yardline,X_speed,Y_speed,X_acceleration,Y_acceleration,age,distance_from_ball,closest_opponent,opponents_in_6,teammates_in_6
0,2017090700,20170907000118,away,46.09,18.493333,1.69,1.13,0.40,81.99,1.620015,496723,45,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,8,72,212,SS,NE,KC,"Foxborough, MA",Outdoor,0,Clear and warm,63.0,77.0,8,SW,True,False,home,1.09,1.687953,-0.083145,1.128632,-0.055594,10480,6.480872,4.59331,3.0,7.0
1,2017090700,20170907000118,away,45.33,20.693333,0.42,1.35,0.01,27.61,1.244420,2495116,45,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,8,75,288,DE,NE,KC,"Foxborough, MA",Outdoor,0,Clear and warm,63.0,77.0,8,SW,True,False,home,0.33,0.397828,0.134657,1.278734,0.432828,10394,4.593310,4.59331,3.0,7.0
2,2017090700,20170907000118,away,46.00,20.133333,1.22,0.59,0.31,3.01,1.174083,2495493,45,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,8,75,270,DE,NE,KC,"Foxborough, MA",Outdoor,0,Clear and warm,63.0,77.0,8,SW,True,False,home,1.00,1.125250,0.471395,0.544178,0.227970,10457,5.448982,4.59331,3.0,7.0
3,2017090700,20170907000118,away,48.54,25.633333,0.42,0.54,0.02,359.77,2.868623,2506353,45,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,8,75,245,ILB,NE,KC,"Foxborough, MA",Outdoor,0,Clear and warm,63.0,77.0,8,SW,True,False,home,3.54,0.113229,-0.404449,0.145580,-0.520006,12709,7.820038,4.59331,3.0,7.0
4,2017090700,20170907000118,away,50.68,17.913333,1.82,2.43,0.16,12.63,1.844638,2530794,45,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,8,72,206,FS,NE,KC,"Foxborough, MA",Outdoor,0,Clear and warm,63.0,77.0,8,SW,True,False,home,5.68,1.752185,-0.492187,2.339456,-0.657151,10980,10.622476,4.59331,3.0,7.0


In [8]:
folds = KFold(5, shuffle=True, random_state=541)

plays = df_train[df_train.has_ball][['PlayId', 
                                     'YardLine', 
                                     'from_yardline', 
                                     'X']].drop_duplicates().reset_index(drop=True)


y1, y2, y3, y4 = create_targets(df_train)

In [9]:
transf_pipe = Pipeline([('trsf', transformation())])

full_train = transf_pipe.fit_transform(df_train)

In [10]:
full_train.head()

,PlayId,X,Y,S,A,Dis,Dir,PlayerHeight,PlayerWeight,from_yardline,X_speed,Y_speed,X_acceleration,Y_acceleration,age,distance_from_ball,closest_opponent,opponents_in_6,teammates_in_6,YardLine,Quarter,Down,Distance,OffenseFormation,DefendersInTheBox,Yards,Temperature,Humidity,HomeScoreBeforePlay,VisitorScoreBeforePlay,poss_avg_height,poss_avg_weight,poss_avg_age,poss_avg_S,poss_avg_A,poss_x_momentum,poss_y_momentum,poss_x_force,poss_y_force,poss_std_X,poss_std_Y,def_avg_height,def_avg_weight,def_avg_age,def_avg_S,def_avg_A,def_x_momentum,def_y_momentum,def_x_force,def_y_force,def_std_X,def_std_Y,tot_x_momenumt,tot_x_force,height_diff,weight_diff,age_diff,X_diff,Y_diff,num_DL,num_LB,num_DB,num_QB,num_RB,num_WR,num_TE,num_OL,OL_diff,OL_TE_diff,run_def
0,20170907000118,41.25,22.803333,3.63,3.35,0.38,0.423417,70,205,3.75,1.491487,3.309436,1.376441,3.054163,9349,0.0,4.593310,3.0,7.0,45,1,3,2,SHOTGUN,6.0,8,63.0,77.0,0,0,74.727273,259.181818,10374.454545,2.106364,1.358182,30394.065885,42824.323203,14808.310917,29092.727988,1.772665,5.855606,73.636364,233.545455,10101.272727,1.314545,1.025455,21967.405111,4867.003640,11406.041625,-783.137540,5.294079,7.014714,8426.660774,3402.269293,1.090909,25.636364,273.181818,-3.521414,-1.159107,2,3,6,1,1,3,1,5,3,4,0
1,20170907000139,48.93,26.173333,3.06,2.41,0.34,5.546656,70,205,4.07,-2.055465,2.266862,-1.618847,1.785339,9349,0.0,4.287773,3.0,7.0,53,1,1,10,SHOTGUN,6.0,3,63.0,77.0,0,0,74.727273,259.181818,10374.454545,2.094545,1.541818,5361.669172,36553.979512,1503.788380,27039.089576,2.001380,6.932502,73.636364,233.545455,10101.272727,1.639091,1.592727,-3627.891830,6256.141431,-2706.292685,4533.856018,5.406292,7.190716,8989.561002,4210.081065,1.090909,25.636364,273.181818,-3.404912,-0.258215,2,3,6,1,1,3,1,5,3,4,0
2,20170907000189,71.34,34.223333,5.77,2.42,0.60,0.838456,70,205,3.66,4.290640,3.857889,1.799541,1.618040,9349,0.0,4.221670,2.0,6.0,75,1,1,10,SINGLEBACK,7.0,5,63.0,77.0,0,0,74.727273,259.181818,10374.454545,3.682727,1.419091,86277.195695,48107.583525,33664.932926,23232.912519,2.115286,6.044208,73.636364,233.545455,10101.272727,3.244545,2.092727,82977.152392,12860.033647,52837.531067,8496.045083,4.720893,6.577620,3300.043303,-19172.598141,1.090909,25.636364,273.181818,-2.605606,-0.533412,2,3,6,1,1,3,1,5,3,4,0
3,20170907000345,104.47,27.973333,4.45,3.20,0.46,6.188239,71,210,3.53,-0.421875,4.429957,-0.303371,3.185587,9808,0.0,4.528002,7.0,9.0,108,1,2,2,JUMBO,9.0,2,63.0,77.0,0,0,76.181818,282.545455,10320.636364,2.141818,0.880909,-39812.029305,39225.794933,-11636.814745,16186.365888,1.898686,3.154869,73.727273,257.454545,10390.181818,1.555455,1.293636,-37637.593645,-4639.830146,-26960.759277,-9328.539063,0.962418,4.505029,-2174.435660,15323.944532,2.454545,25.090909,-69.545455,0.936268,-1.350160,4,4,3,1,2,0,2,6,2,4,1
4,20170907000395,29.99,27.120000,3.90,2.53,0.44,5.097758,71,216,5.01,-3.613974,1.466013,-2.344450,0.951029,8069,0.0,4.288088,3.0,6.0,35,1,1,10,SHOTGUN,7.0,7,63.0,77.0,7,0,76.909091,268.454545,9732.818182,2.644545,1.620000,-31427.161922,20004.068984,-13921.201825,13577.076191,2.056883,7.423977,73.181818,242.454545,9967.181818,2.322727,2.121818,-54049.085285,-11151.985380,-50459.233839,-5737.783040,5.391251,7.864325,22621.923362,36538.032014,3.727273,26.000000,-234.363636,-3.334368,-0.440348,3,2,6,1,1,1,3,5,2,5,0


In [11]:
def get_crps(train, inf_pred, y):
    y_pred = np.zeros((len(train),199))
    y_true = np.zeros((len(train),199))

    for i,p in enumerate(inf_preds):
        p += 99
        for j in range(199):
            if j >= p + 10:
                y_pred[i][j] = 1.0
            elif j >= p - 10:
                y_pred[i][j] = (j + 10 - p) * 0.05

    for i,p in enumerate(y):
        p += 99
        for j in range(199):
            if j >= p:
                y_true[i][j]=1.0
                
    return np.sum(np.power(y_pred - y_true, 2)) / (199 * (len(train)))

In [12]:
warnings.filterwarnings("ignore", 
                        message="The dummies in this set do not match the ones in the train set, we corrected the issue.")

In [14]:
models = [('lasso', Lasso(alpha=0.01)), ('ridge', Ridge()), ('sgd', SGDRegressor()), 
          ('forest', RandomForestRegressor(n_estimators=100, n_jobs=-1, max_depth=10, 
                                           criterion='mae', max_features='sqrt')), 
          ('xtree', ExtraTreesRegressor(n_estimators=200, n_jobs=-1, max_depth=10,
                                        criterion='mae', max_features='sqrt')), 
          ('svr', SVR(gamma='auto')),
          ('xgb', xgb.XGBRegressor(n_estimators=600, objective='reg:squarederror', n_jobs=-1)), 
          ('lgb', lgb.LGBMRegressor(n_estimators=600, n_jobs=-1))]

mod_name = []
rmse_train_simple = []
mae_train_simple = []
crps_train_simple = []

rmse_train_tot = []
mae_train_tot = []
crps_train_tot = []

mae_train_prcgained = []
rmse_train_prcgained = []
crps_train_prcgained = []

rmse_train_prctot = []
mae_train_prctot = []
crps_train_prctot = []

for model in models:
    
    train = full_train.copy()
    mod_play = plays.copy()
    del train['PlayId']
    del train['Yards']
    print(model[0])
    mod_name.append(model[0])
    
    model_pipe = Pipeline([('dummifier', dummify(drop_first=True)),
                           ('Imputer', df_imputer()),
                           ('scl', df_scaler(method='standard'))] + [model])
            
    %time inf_preds = cv_score(train, y1, folds, model_pipe)
    
    mod_play['simple'] = inf_preds
    mod_play.loc[mod_play.simple > (110 - mod_play.YardLine), 'simple'] = (110 - mod_play.YardLine)
    inf_preds = mod_play['simple']
    
    avg_preds = inf_preds / 4
    
    crps = get_crps(train, inf_preds, y1)

    rmse_train_simple.append(np.sqrt(mean_squared_error(y1, inf_preds)))
    mae_train_simple.append(mean_absolute_error(y1, inf_preds))
    crps_train_simple.append(crps)
    
    print(f'\tTrain set RMSE: {round(np.sqrt(mean_squared_error(y1, inf_preds)), 4)}')
    print(f'\tTrain set MAE: {round(mean_absolute_error(y1, inf_preds), 4)}')
    print(f'\tTrain set CRPS: {round(crps, 4)}')
    
    print('- -'*20)
    
    %time inf_preds = cv_score(train, y2, folds, model_pipe)
    
    mod_play['total'] = inf_preds
    mod_play['total'] = mod_play['total'] - mod_play['from_yardline']
    inf_preds = mod_play['total']
    
    avg_preds += inf_preds / 4
    
    crps = get_crps(train, inf_preds, y1)
    
    rmse_train_tot.append(np.sqrt(mean_squared_error(y1, inf_preds)))
    mae_train_tot.append(mean_absolute_error(y1, inf_preds))
    crps_train_tot.append(crps)
    
    print(f'\tTrain set RMSE full distance: {round(np.sqrt(mean_squared_error(y1, inf_preds)), 4)}')
    print(f'\tTrain set MAE full distance: {round(mean_absolute_error(y1, inf_preds), 4)}')
    print(f'\tTrain set CRPS full distance: {round(crps, 4)}')
    
    print('- -'*20)
    
    %time inf_preds = cv_score(train, y3, folds, model_pipe)
    
    mod_play['per_gain'] = inf_preds
    mod_play['per_gain'] = mod_play['per_gain'] * (110 - mod_play['YardLine'])
    inf_preds = mod_play['per_gain']
    
    avg_preds += inf_preds / 4
    
    crps = get_crps(train, inf_preds, y1)
    
    rmse_train_prcgained.append(np.sqrt(mean_squared_error(y1, inf_preds)))
    mae_train_prcgained.append(mean_absolute_error(y1, inf_preds))
    crps_train_prcgained.append(crps)
    
    print(f'\tTrain set RMSE percentage gained: {round(np.sqrt(mean_squared_error(y1, inf_preds)), 4)}')
    print(f'\tTrain set MAE percentage gained: {round(mean_absolute_error(y1, inf_preds), 4)}')
    print(f'\tTrain set CRPS percentage gained: {round(crps, 4)}')
    
    print('- -'*20)
    
    %time inf_preds = cv_score(train, y4, folds, model_pipe)
    
    mod_play['per_tot'] = inf_preds
    mod_play['per_tot'] = mod_play['per_tot'] * (110 - mod_play['X']) - mod_play['from_yardline']
    inf_preds = mod_play['per_tot']
    
    avg_preds += inf_preds / 4
    
    crps = get_crps(train, inf_preds, y1)
    
    rmse_train_prctot.append(np.sqrt(mean_squared_error(y1, inf_preds)))
    mae_train_prctot.append(mean_absolute_error(y1, inf_preds))
    crps_train_prctot.append(crps)
    
    print(f'\tTrain set RMSE full distance percentage: {round(np.sqrt(mean_squared_error(y1, inf_preds)), 4)}')
    print(f'\tTrain set MAE full distance percentage: {round(mean_absolute_error(y1, inf_preds), 4)}')
    print(f'\tTrain set CRPS full distance percentage: {round(crps, 4)}')
    
    print('- -'*20)
    
    print(f'\tTrain set RMSE average: {round(np.sqrt(mean_squared_error(y1, avg_preds)), 4)}')
    print(f'\tTrain set MAE average: {round(mean_absolute_error(y1, avg_preds), 4)}')
    print(f'\tTrain set CRPS average: {round(crps, 4)}')

    print('_'*60)
    print('\n')
    
    
results = pd.DataFrame({'model_name': mod_name, 
                        'rmse_train': rmse_train_simple, 
                        'mae_train': mae_train_simple, 
                        'crps_train': crps_train_simple, 
                        'rmse_tot_dist': rmse_train_tot, 
                        'mae_tot_dist': mae_train_tot, 
                        'crps_tot_dist': crps_train_tot,
                        'rmse_perc_gained': rmse_train_prcgained, 
                        'mae_perc_gained': mae_train_prcgained, 
                        'crps_perc_gained': crps_train_prcgained,
                        'rmse_perc_tot': rmse_train_prctot, 
                        'mae_perc_tot': mae_train_prctot, 
                        'crps_perc_tot': crps_train_prctot,})

results

lasso
CPU times: user 11.6 s, sys: 4.69 s, total: 16.3 s
Wall time: 2.87 s
	Train set RMSE: 6.1953
	Train set MAE: 3.6085
	Train set CRPS: 0.0151
- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -
CPU times: user 2.38 s, sys: 1.71 s, total: 4.09 s
Wall time: 815 ms
	Train set RMSE full distance: 6.2016
	Train set MAE full distance: 3.6283
	Train set CRPS full distance: 0.0151
- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -
CPU times: user 1.8 s, sys: 1.2 s, total: 3.01 s
Wall time: 606 ms
	Train set RMSE percentage gained: 6.4252
	Train set MAE percentage gained: 4.0083
	Train set CRPS percentage gained: 0.0159
- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -
CPU times: user 11.7 s, sys: 4.69 s, total: 16.3 s
Wall time: 3.05 s
	Train set RMSE full distance percentage: 7.0952
	Train set MAE full distance percentage: 4.753
	Train set CRPS full distance percentage: 0.018
- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -
	Train set RMSE average

CPU times: user 42.8 s, sys: 191 ms, total: 43 s
Wall time: 7.82 s
	Train set RMSE full distance: 6.3022
	Train set MAE full distance: 3.766
	Train set CRPS full distance: 0.0155
- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -
CPU times: user 44.2 s, sys: 195 ms, total: 44.4 s
Wall time: 8.15 s
	Train set RMSE percentage gained: 6.3453
	Train set MAE percentage gained: 3.8159
	Train set CRPS percentage gained: 0.0157
- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -
CPU times: user 48 s, sys: 264 ms, total: 48.3 s
Wall time: 8.95 s
	Train set RMSE full distance percentage: 6.3135
	Train set MAE full distance percentage: 3.7836
	Train set CRPS full distance percentage: 0.0156
- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -
	Train set RMSE average: 6.1977
	Train set MAE average: 3.6607
	Train set CRPS average: 0.0156
____________________________________________________________




,model_name,rmse_train,mae_train,crps_train,rmse_tot_dist,mae_tot_dist,crps_tot_dist,rmse_perc_gained,mae_perc_gained,crps_perc_gained,rmse_perc_tot,mae_perc_tot,crps_perc_tot
0,lasso,6.195320,3.608494,0.015083,6.201601,3.628272,0.015107,6.425197,4.008340,0.015901,7.095192,4.753014,0.017958
1,ridge,6.197739,3.612972,0.015092,6.204245,3.633126,0.015118,6.673772,4.242491,0.016583,7.342108,4.918254,0.018617
2,sgd,6.232527,3.664974,0.015216,6.235412,3.693693,0.015239,6.974714,4.537485,0.017494,7.389287,4.995095,0.018843
3,forest,6.328564,3.434230,0.014910,6.312379,3.423949,0.014879,6.310501,3.492545,0.014994,6.386520,3.627800,0.015292
4,xtree,6.446957,3.533755,0.015149,6.441889,3.526110,0.015124,6.406060,3.573129,0.015168,6.486766,3.785567,0.015552
5,svr,6.306508,3.366796,0.014777,6.312265,3.376336,0.014791,6.926441,4.568628,0.017821,7.014546,4.703873,0.018243
6,xgb,6.234128,3.637225,0.015208,6.252055,3.657938,0.015256,6.353324,3.743068,0.015474,6.246691,3.689854,0.015324
7,lgb,6.300628,3.761901,0.015518,6.302162,3.765957,0.015514,6.345299,3.815941,0.015655,6.313524,3.783597,0.015569


In [15]:
results.corr()

,rmse_train,mae_train,crps_train,rmse_tot_dist,mae_tot_dist,crps_tot_dist,rmse_perc_gained,mae_perc_gained,crps_perc_gained,rmse_perc_tot,mae_perc_tot,crps_perc_tot
rmse_train,1.000000,-0.378041,-0.072138,0.994887,-0.448263,-0.168038,-0.255132,-0.485404,-0.373852,-0.530787,-0.593399,-0.572626
mae_train,-0.378041,1.000000,0.943381,-0.356759,0.995489,0.966786,-0.208970,-0.058729,-0.158268,-0.035456,-0.002160,-0.048201
crps_train,-0.072138,0.943381,1.000000,-0.049628,0.912626,0.992765,-0.334340,-0.236336,-0.298549,-0.282434,-0.254111,-0.292410
rmse_tot_dist,0.994887,-0.356759,-0.049628,1.000000,-0.423990,-0.138614,-0.245151,-0.467008,-0.355688,-0.544073,-0.595136,-0.575337
mae_tot_dist,-0.448263,0.995489,0.912626,-0.423990,1.000000,0.946194,-0.142122,0.017028,-0.087146,0.033880,0.072761,0.026984
crps_tot_dist,-0.168038,0.966786,0.992765,-0.138614,0.946194,1.000000,-0.287636,-0.169036,-0.239619,-0.230783,-0.190916,-0.230545
rmse_perc_gained,-0.255132,-0.208970,-0.334340,-0.245151,-0.142122,-0.287636,1.000000,0.943465,0.966682,0.793142,0.806917,0.840728
mae_perc_gained,-0.485404,-0.058729,-0.236336,-0.467008,0.017028,-0.169036,0.943465,1.000000,0.988072,0.828638,0.886714,0.908229
crps_perc_gained,-0.373852,-0.158268,-0.298549,-0.355688,-0.087146,-0.239619,0.966682,0.988072,1.000000,0.768683,0.823955,0.854898
rmse_perc_tot,-0.530787,-0.035456,-0.282434,-0.544073,0.033880,-0.230783,0.793142,0.828638,0.768683,1.000000,0.983449,0.981733


In [16]:
results.sort_values(by='rmse_train')

,model_name,rmse_train,mae_train,crps_train,rmse_tot_dist,mae_tot_dist,crps_tot_dist,rmse_perc_gained,mae_perc_gained,crps_perc_gained,rmse_perc_tot,mae_perc_tot,crps_perc_tot
0,lasso,6.195320,3.608494,0.015083,6.201601,3.628272,0.015107,6.425197,4.008340,0.015901,7.095192,4.753014,0.017958
1,ridge,6.197739,3.612972,0.015092,6.204245,3.633126,0.015118,6.673772,4.242491,0.016583,7.342108,4.918254,0.018617
2,sgd,6.232527,3.664974,0.015216,6.235412,3.693693,0.015239,6.974714,4.537485,0.017494,7.389287,4.995095,0.018843
6,xgb,6.234128,3.637225,0.015208,6.252055,3.657938,0.015256,6.353324,3.743068,0.015474,6.246691,3.689854,0.015324
7,lgb,6.300628,3.761901,0.015518,6.302162,3.765957,0.015514,6.345299,3.815941,0.015655,6.313524,3.783597,0.015569
5,svr,6.306508,3.366796,0.014777,6.312265,3.376336,0.014791,6.926441,4.568628,0.017821,7.014546,4.703873,0.018243
3,forest,6.328564,3.434230,0.014910,6.312379,3.423949,0.014879,6.310501,3.492545,0.014994,6.386520,3.627800,0.015292
4,xtree,6.446957,3.533755,0.015149,6.441889,3.526110,0.015124,6.406060,3.573129,0.015168,6.486766,3.785567,0.015552


In [17]:
results.sort_values(by='mae_train')

,model_name,rmse_train,mae_train,crps_train,rmse_tot_dist,mae_tot_dist,crps_tot_dist,rmse_perc_gained,mae_perc_gained,crps_perc_gained,rmse_perc_tot,mae_perc_tot,crps_perc_tot
5,svr,6.306508,3.366796,0.014777,6.312265,3.376336,0.014791,6.926441,4.568628,0.017821,7.014546,4.703873,0.018243
3,forest,6.328564,3.434230,0.014910,6.312379,3.423949,0.014879,6.310501,3.492545,0.014994,6.386520,3.627800,0.015292
4,xtree,6.446957,3.533755,0.015149,6.441889,3.526110,0.015124,6.406060,3.573129,0.015168,6.486766,3.785567,0.015552
0,lasso,6.195320,3.608494,0.015083,6.201601,3.628272,0.015107,6.425197,4.008340,0.015901,7.095192,4.753014,0.017958
1,ridge,6.197739,3.612972,0.015092,6.204245,3.633126,0.015118,6.673772,4.242491,0.016583,7.342108,4.918254,0.018617
6,xgb,6.234128,3.637225,0.015208,6.252055,3.657938,0.015256,6.353324,3.743068,0.015474,6.246691,3.689854,0.015324
2,sgd,6.232527,3.664974,0.015216,6.235412,3.693693,0.015239,6.974714,4.537485,0.017494,7.389287,4.995095,0.018843
7,lgb,6.300628,3.761901,0.015518,6.302162,3.765957,0.015514,6.345299,3.815941,0.015655,6.313524,3.783597,0.015569


In [18]:
results.sort_values(by='crps_train')

,model_name,rmse_train,mae_train,crps_train,rmse_tot_dist,mae_tot_dist,crps_tot_dist,rmse_perc_gained,mae_perc_gained,crps_perc_gained,rmse_perc_tot,mae_perc_tot,crps_perc_tot
5,svr,6.306508,3.366796,0.014777,6.312265,3.376336,0.014791,6.926441,4.568628,0.017821,7.014546,4.703873,0.018243
3,forest,6.328564,3.434230,0.014910,6.312379,3.423949,0.014879,6.310501,3.492545,0.014994,6.386520,3.627800,0.015292
0,lasso,6.195320,3.608494,0.015083,6.201601,3.628272,0.015107,6.425197,4.008340,0.015901,7.095192,4.753014,0.017958
1,ridge,6.197739,3.612972,0.015092,6.204245,3.633126,0.015118,6.673772,4.242491,0.016583,7.342108,4.918254,0.018617
4,xtree,6.446957,3.533755,0.015149,6.441889,3.526110,0.015124,6.406060,3.573129,0.015168,6.486766,3.785567,0.015552
6,xgb,6.234128,3.637225,0.015208,6.252055,3.657938,0.015256,6.353324,3.743068,0.015474,6.246691,3.689854,0.015324
2,sgd,6.232527,3.664974,0.015216,6.235412,3.693693,0.015239,6.974714,4.537485,0.017494,7.389287,4.995095,0.018843
7,lgb,6.300628,3.761901,0.015518,6.302162,3.765957,0.015514,6.345299,3.815941,0.015655,6.313524,3.783597,0.015569


In [19]:
results.sort_values(by='mae_tot_dist')

,model_name,rmse_train,mae_train,crps_train,rmse_tot_dist,mae_tot_dist,crps_tot_dist,rmse_perc_gained,mae_perc_gained,crps_perc_gained,rmse_perc_tot,mae_perc_tot,crps_perc_tot
5,svr,6.306508,3.366796,0.014777,6.312265,3.376336,0.014791,6.926441,4.568628,0.017821,7.014546,4.703873,0.018243
3,forest,6.328564,3.434230,0.014910,6.312379,3.423949,0.014879,6.310501,3.492545,0.014994,6.386520,3.627800,0.015292
4,xtree,6.446957,3.533755,0.015149,6.441889,3.526110,0.015124,6.406060,3.573129,0.015168,6.486766,3.785567,0.015552
0,lasso,6.195320,3.608494,0.015083,6.201601,3.628272,0.015107,6.425197,4.008340,0.015901,7.095192,4.753014,0.017958
1,ridge,6.197739,3.612972,0.015092,6.204245,3.633126,0.015118,6.673772,4.242491,0.016583,7.342108,4.918254,0.018617
6,xgb,6.234128,3.637225,0.015208,6.252055,3.657938,0.015256,6.353324,3.743068,0.015474,6.246691,3.689854,0.015324
2,sgd,6.232527,3.664974,0.015216,6.235412,3.693693,0.015239,6.974714,4.537485,0.017494,7.389287,4.995095,0.018843
7,lgb,6.300628,3.761901,0.015518,6.302162,3.765957,0.015514,6.345299,3.815941,0.015655,6.313524,3.783597,0.015569


In [20]:
results.sort_values(by='mae_perc_gained')

,model_name,rmse_train,mae_train,crps_train,rmse_tot_dist,mae_tot_dist,crps_tot_dist,rmse_perc_gained,mae_perc_gained,crps_perc_gained,rmse_perc_tot,mae_perc_tot,crps_perc_tot
3,forest,6.328564,3.434230,0.014910,6.312379,3.423949,0.014879,6.310501,3.492545,0.014994,6.386520,3.627800,0.015292
4,xtree,6.446957,3.533755,0.015149,6.441889,3.526110,0.015124,6.406060,3.573129,0.015168,6.486766,3.785567,0.015552
6,xgb,6.234128,3.637225,0.015208,6.252055,3.657938,0.015256,6.353324,3.743068,0.015474,6.246691,3.689854,0.015324
7,lgb,6.300628,3.761901,0.015518,6.302162,3.765957,0.015514,6.345299,3.815941,0.015655,6.313524,3.783597,0.015569
0,lasso,6.195320,3.608494,0.015083,6.201601,3.628272,0.015107,6.425197,4.008340,0.015901,7.095192,4.753014,0.017958
1,ridge,6.197739,3.612972,0.015092,6.204245,3.633126,0.015118,6.673772,4.242491,0.016583,7.342108,4.918254,0.018617
2,sgd,6.232527,3.664974,0.015216,6.235412,3.693693,0.015239,6.974714,4.537485,0.017494,7.389287,4.995095,0.018843
5,svr,6.306508,3.366796,0.014777,6.312265,3.376336,0.014791,6.926441,4.568628,0.017821,7.014546,4.703873,0.018243


In [21]:
results.sort_values(by='mae_perc_tot')

,model_name,rmse_train,mae_train,crps_train,rmse_tot_dist,mae_tot_dist,crps_tot_dist,rmse_perc_gained,mae_perc_gained,crps_perc_gained,rmse_perc_tot,mae_perc_tot,crps_perc_tot
3,forest,6.328564,3.434230,0.014910,6.312379,3.423949,0.014879,6.310501,3.492545,0.014994,6.386520,3.627800,0.015292
6,xgb,6.234128,3.637225,0.015208,6.252055,3.657938,0.015256,6.353324,3.743068,0.015474,6.246691,3.689854,0.015324
7,lgb,6.300628,3.761901,0.015518,6.302162,3.765957,0.015514,6.345299,3.815941,0.015655,6.313524,3.783597,0.015569
4,xtree,6.446957,3.533755,0.015149,6.441889,3.526110,0.015124,6.406060,3.573129,0.015168,6.486766,3.785567,0.015552
5,svr,6.306508,3.366796,0.014777,6.312265,3.376336,0.014791,6.926441,4.568628,0.017821,7.014546,4.703873,0.018243
0,lasso,6.195320,3.608494,0.015083,6.201601,3.628272,0.015107,6.425197,4.008340,0.015901,7.095192,4.753014,0.017958
1,ridge,6.197739,3.612972,0.015092,6.204245,3.633126,0.015118,6.673772,4.242491,0.016583,7.342108,4.918254,0.018617
2,sgd,6.232527,3.664974,0.015216,6.235412,3.693693,0.015239,6.974714,4.537485,0.017494,7.389287,4.995095,0.018843
